In [1]:
import cv2
import numpy as np

# YOLOv3 608 모델 로드
yolo_net = cv2.dnn.readNet("./yolo_file/yolov3.weights", "./yolo_file/yolov3.cfg")

# YOLO 레이어 가져오기
output_layers = [yolo_net.getLayerNames()[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

# 클래스 이름 로드 (coco.names 파일)
classes = []
with open("./yolo_file/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def process_frame(frame):
    # YOLO 모델을 사용하여 객체를 감지하는 함수
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    outputs = yolo_net.forward(output_layers)
    return outputs

def get_objects_from_frame(frame):
    # 프레임에서 객체를 추출하는 함수
    outputs = process_frame(frame)
    detected_objects = []
    boxes = []  # 객체의 바운딩 박스를 저장할 리스트
    confidences = []  # 객체의 신뢰도를 저장할 리스트
    class_ids = []  # 클래스 ID 저장
    
    height, width, _ = frame.shape
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]  # 확률 정보 (각 클래스의 확률)
            class_id = np.argmax(scores)  # 확률이 가장 높은 클래스의 인덱스
            confidence = scores[class_id]  # 해당 클래스의 확률 (신뢰도)

            if confidence > 0.5:  # 신뢰도가 0.5 이상인 객체만 추출
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 바운딩 박스의 좌상단 코너
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    # Non-maxima suppression을 사용하여 겹치는 박스를 제거
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    for i in indices.flatten():
        box = boxes[i]
        x, y, w, h = box
        label = str(classes[class_ids[i]])
        color = (0, 255, 0)  # 객체의 색깔 (초록색)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return frame, detected_objects

# 프레임에서 객체를 감지하는 예시
frame = cv2.imread("./bb_1_180114_pedestrian_241_22483.mp4_frame_66.jpg")
frame_with_objects, detected_objects = get_objects_from_frame(frame)

# 결과 이미지 보여주기
cv2.imshow("Detected Objects", frame_with_objects)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 감지된 객체들 출력
print(detected_objects)


[]
